In [354]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [406]:
df=pd.read_csv('./data/train.csv')

In [407]:
df.corr()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
Age,1.000000,0.068723,0.130421,0.033133,0.123970,0.101007,-0.075026
RoomService,0.068723,1.000000,-0.015889,0.054480,0.010080,-0.019581,-0.244611
FoodCourt,0.130421,-0.015889,1.000000,-0.014228,0.221891,0.227995,0.046566
ShoppingMall,0.033133,0.054480,-0.014228,1.000000,0.013879,-0.007322,0.010141
Spa,0.123970,0.010080,0.221891,0.013879,1.000000,0.153821,-0.221131
VRDeck,0.101007,-0.019581,0.227995,-0.007322,0.153821,1.000000,-0.207075
Transported,-0.075026,-0.244611,0.046566,0.010141,-0.221131,-0.207075,1.000000


In [408]:
df['Transported'].value_counts()

True     4378
False    4315
Name: Transported, dtype: int64

In [357]:
df.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [358]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [359]:
df.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [360]:
def data_preprocessing_pipeline(df):
    df=df.drop('PassengerId',axis=1)
    df=df.drop('Name',axis=1)
    
    df['Cabin']=df['Cabin'].str.split('/')
    deck=[]
    number=[]
    side=[]
    for item in df['Cabin']:
        try:
            deck.append(item[0])
            number.append(int(item[1]))
            side.append(item[2])
        except:
            deck.append(float('nan'))
            number.append(float('nan'))
            side.append(float('nan'))
    
    df['cabin_deck']=pd.Series(deck)
    df['cabin_number']=pd.Series(number)
    df['cabin_side']=pd.Series(side)
    df[['cabin_deck','cabin_side']]=df[['cabin_deck','cabin_side']].fillna('Other')
    df['cabin_number']=df['cabin_number'].fillna(-1)
    
    df=df.drop('Cabin',axis=1)
    
    col_list=['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
    df[col_list]=df[col_list].fillna(0)
    
    df['VIP']=df['VIP'].fillna(False)
    
    
    n=df['Destination'].isna().sum()
    
    arr=np.random.random(size=(n,))
    lst=["" for i in range(n)]
    ser=pd.Series(lst)
    ser[arr<0.680433]='TRAPPIST-1e'
    ser[arr<0.680433+0.207063]='55 Cancri e'
    ser[arr>=0.680433+0.207063]='PSO J318.5-22'
    df['Destination'].iloc[df['Destination'].isna()]=ser
    
    n=df['HomePlanet'].isna().sum()
    arr=np.random.random(size=(n,))
    lst=["" for i in range(n)]
    ser=pd.Series(lst)
    ser[arr<0.529391]='Earth'
    ser[arr<0.529391+0.245140]='Europa'
    ser[arr>=0.529391+0.245140]='Mars'
    df['HomePlanet'].iloc[df['HomePlanet'].isna()]=ser
    
    avg_age=df.groupby('HomePlanet').mean()['Age']
    planet_list=['Earth','Europa','Mars']
    
    for planet in planet_list:
        val=avg_age[planet]
        df['Age'][(df['Age'].isna() * df['HomePlanet']==planet)]=df['Age'][(df['Age'].isna() * df['HomePlanet']==planet)].fillna(val)
    
    df['CryoSleep']=df['CryoSleep'].fillna(False)
    
    df=pd.get_dummies(df,drop_first=True)
    
    df=df.astype('float')
    
    return df

In [361]:
df=data_preprocessing_pipeline(df)

C:\Users\Advait_PC\anaconda3\envs\tfDeepLearningEnv\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\ADVAIT~1\AppData\Local\Temp/ipykernel_29472/3304017042.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'][(df['Age'].isna() * df['HomePlanet']==planet)]=df['Age'][(df['Age'].isna() * df['HomePlanet']==planet)].fillna(val)


In [362]:
x=df.drop('Transported',axis=1).values
y=df['Transported'].values

In [368]:
y.shape

(8693,)

In [367]:
x.shape

(8693, 23)

In [366]:
from sklearn.model_selection import train_test_split

In [369]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [370]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [371]:
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [372]:
from sklearn.ensemble import RandomForestClassifier

In [373]:
model = RandomForestClassifier()

In [374]:
model.fit(X_train,y_train)

RandomForestClassifier()

In [376]:
y_pred=model.predict(X_test)

In [378]:
from sklearn.metrics import classification_report

In [380]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.78      0.81      0.79       443
         1.0       0.79      0.76      0.78       427

    accuracy                           0.79       870
   macro avg       0.79      0.78      0.78       870
weighted avg       0.79      0.79      0.78       870



In [392]:
df_test=pd.read_csv('./data/test.csv')
idx=df_test['PassengerId']

In [393]:
df_test=data_preprocessing_pipeline(df_test)

C:\Users\Advait_PC\anaconda3\envs\tfDeepLearningEnv\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\ADVAIT~1\AppData\Local\Temp/ipykernel_29472/3304017042.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'][(df['Age'].isna() * df['HomePlanet']==planet)]=df['Age'][(df['Age'].isna() * df['HomePlanet']==planet)].fillna(val)


In [394]:
X_test=df_test.values

In [395]:
X_test=scaler.transform(X_test)

In [396]:
y_pred=model.predict(X_test)

In [397]:
y_pred

array([1., 0., 1., ..., 1., 1., 0.])

In [398]:
y_pred=y_pred.astype('bool')

In [403]:
df=pd.DataFrame(y_pred,index=idx.values,columns=['Transported'])
df.index.name='PassengerId'

In [405]:
df.to_csv('submission.csv')